In [ ]:
# Parameters. Set defaults here.
# Times Square replaces this cell with the user's parameters.
record_limit = '99'

<a class="anchor" id="imports"></a>
## Imports and General Setup

In [ ]:
# Only use packages available in the Rubin Science Platform
import requests
from collections import defaultdict
import pandas as pd
from pprint import pp
from urllib.parse import urlencode
from IPython.display import FileLink, display_markdown
from matplotlib import pyplot as plt
import os
from datetime import datetime, date

In [ ]:
env = 'usdf_dev'  # usdf-dev, tucson, slac, summit
log_name = 'narrativelog'
log = log_name
limit = int(record_limit)
response_timeout = 3.05  # seconds, how long to wait for connection
read_timeout = 20  # seconds

timeout = (float(response_timeout), float(read_timeout))

summit = 'https://summit-lsp.lsst.codes'
usdf = 'https://usdf-rsp-dev.slac.stanford.edu'
tucson = 'https://tucson-teststand.lsst.codes'
server = os.environ.get('EXTERNAL_INSTANCE_URL', usdf)
service = f'{server}/{log}'
service

<a class="anchor" id="setup_source"></a>
## Setup Source

In [ ]:
ignore_fields = set(['tags', 'urls', 'message_text', 'id', 'date_added', 
                     'obs_id', 'day_obs', 'seq_num', 'parent_id', 'user_id',
                     'date_invalidated', 'date_begin', 'date_end',
                     'time_lost', # float
                     #'systems','subsystems','cscs',  # values are lists, special handling
                    ])

In [ ]:
# For Times Square, comment out next line and past next cell with contents of local python file.
from lsst.ts.logging_and_reporting.logrep_utils import NarrativelogAdapter
# Once our logrep package has been installed in RSP, we can use the simpler "import"

In [ ]:
# Paste contents of logrep_utils.py here

<a class="anchor" id="get_records"></a>
## Get Records

In [ ]:
service_adapter = NarrativelogAdapter(server_url=server)
try:
    recs = service_adapter.get_messages()
except Exception as err:
    recs = []
    msg = f'ERROR getting records from {server=}: {err=}'
    raise Exception(msg)

flds = set(recs[0].keys())
facflds = flds - ignore_fields
# facets(field) = set(value-1, value-2, ...)
facets = {fld: set([str(r[fld])
                    for r in recs if not isinstance(r[fld], list)]) 
                    for fld in facflds}

print(f'Retrieved {len(recs) } records, each with {len(flds)} fields.')

In [ ]:
date_tl = service_adapter.get_timelost()

<a class="anchor" id="table"></a>
## Tables of (mostly raw) results

### Fields names provided in records from log.

In [ ]:
pd.DataFrame(flds, columns=['Field Name'])

### Facets from log records.
A *facet* is the set all of values found for a field in the retrieved records. Facets are only calculated for some fields.

In [ ]:
display(pd.DataFrame.from_dict(facets, orient='index'))
facets

### Show Retrieved Records

In [ ]:
ignore_fields = ['id']
new_column_names = dict(message_text='message',
                        primary_software_components='PSC',
                        primary_hardware_components='PHC',
                       )

df = pd.DataFrame(recs).rename(columns=new_column_names)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df.loc[:, ~df.columns.isin(ignore_fields)])

### Table of selected log record fields.
Table can be retrieved as CSV file for local use.

In [ ]:
cols = ['date_added', 'time_lost', 'time_lost_type']
df = pd.DataFrame(recs)[cols]

# Allow download of CSV version of DataFrame
csvfile = 'tl.csv'
df.to_csv(csvfile)
myfile = FileLink(csvfile)
print('Table available as CSV file: ')
display(myfile)
df

In [ ]:
cols = ['message_text','tags','user_id', 'components','date_end']
df = pd.DataFrame(recs, columns=cols)
df

<a class="anchor" id="plot"></a>
## Plots from log (per day)

In [ ]:
display_markdown(f'### Date vs Time Lost', raw=True)
display_markdown(f'(Should rollup per day)', raw=True)
x = date_tl.keys()
y = date_tl.values()

df = pd.DataFrame(dict(day=x,time_lost=y))
df.plot.bar(x='day', y='time_lost', title=f'Time Lost')
display(df)

<a class="anchor" id="raw_analysis"></a>
## Raw Content Analysis

### Example of one record

In [ ]:
rec = recs[-1]

msg = rec["message_text"]
md = f'Message text from log:\n> {msg}'
display_markdown(md, raw=True)

md = f'One full record (the last one retrieved):\n> {rec}'
display_markdown(md, raw=True)

display(rec)

<a class="anchor" id="elicitation"></a>
## Stakeholder Elicitation